In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_rows', 70)

In [ ]:
folder = "BEH"
files = [os.path.join(folder, f) for f in os.listdir(folder)]
#files = ['BEH\\beh_B_RS0303.csv']

In [ ]:
pd.read_csv(files[0])[30:50]

In [ ]:
# creating column list
columns_list = ["names", "mean RT", "SD RT", "mean RT no post stop", "SD RT no post stop", "% error", 
               "mean SSD", "mean SSD US", "inhibition_rate", "bound_rt", "SSRT", "mean RT US", "SD RT US", 
                "mean RT post US", "SD RT post US", "mean RT post SS", "SD RT post SS"]

columns_list_stops = ["_mean SSD", "_mean SSD US", "_inhibition_rate", "_bound_rt", "_SSRT", "_mean RT US", 
                      "_SD RT US", 
                      "_mean RT post US", "_SD RT post US", "_mean RT post SS", "_SD RT post SS"]


stops_types = ["NEG", "POS", "NEU"]
for s_t in stops_types:
    columns_list_stop_type = [s_t + elem for elem in columns_list_stops]
    columns_list += columns_list_stop_type
    
# creating data frame with all data
beh_info = pd.DataFrame(columns=columns_list)

for i, f in enumerate(files):
    name = f.split("\\")[1]
    beh_info.at[i, "names"] = name
    #load data
    data = pd.read_csv(f)[30:]
    #-------------------- Zmienne związane z reakcją na bodziec Go -------------------#
    # mean RT
    rt = data[data["ST_type"].isnull()]
    rt_good = rt[rt["RE_key"] == rt["RE_true"]]
    beh_info.at[i, "mean RT"] = rt_good["RE_time"].mean()
    
    # SD RT
    beh_info.at[i, "SD RT"] = rt_good["RE_time"].std()
    
    # mean RT no post stop, SD RT no post stop
    last = None
    rt_no_post_stop = []
    
    for _, row in data.iterrows():
        if  last is not None and \
            last["ST_type"] != "text" and \
            row["ST_type"] != "text" and \
            last["RE_key"] == last["RE_true"] and \
            row["RE_key"] == row["RE_true"]:
            rt_no_post_stop.append(row["RE_time"])
        last = row
    
    # mean RT no post stop
    beh_info.at[i, "mean RT no post stop"] = np.mean(rt_no_post_stop)
    
    # SD RT no post stop
    beh_info.at[i, "SD RT no post stop"] = np.std(rt_no_post_stop, ddof=1)
    
    # % error
    rt_len = len(rt)
    rt_error = rt[rt["RE_time"] > 0]
    rt_error = rt_error[rt_error["RE_key"] != rt_error["RE_true"]]
    beh_info.at[i, "% error"] = len(rt_error)/rt_len * 100
    
    # mean SSD
    stops = data[data["ST_wait_time"] > 0]
    ssd = stops["ST_wait_time"].mean()
    beh_info.at[i, "mean SSD"] = ssd
    
    # mean SSD US
    us = stops[stops["RE_time"] > 0]
    beh_info.at[i, "mean SSD US"] = us["ST_wait_time"].mean()
    
    # inhibition_rate
    ss = stops[stops["RE_key"].isnull()]
    ir = len(ss)/len(stops)*100
    beh_info.at[i, "inhibition_rate"] = ir
    
    # bound_rt
    us_perc = 1 - ir/100
    rt_good_sorted = sorted(rt_good["RE_time"])
    bound_rt_index = round(len(rt_good_sorted) * us_perc)
    bound_rt = rt_good_sorted[bound_rt_index-1]
    beh_info.at[i, "bound_rt"] = bound_rt
    
    # SSRT
    beh_info.at[i, "SSRT"] = bound_rt - ssd
    
    # mean RT US
    us = stops[stops["RE_time"] > 0]
    beh_info.at[i, "mean RT US"] = us["RE_time"].mean()

    # SD RT US
    beh_info.at[i, "SD RT US"] = us["RE_time"].std()

    # mean RT post US, SD RT post US, mean RT post SS, SD RT post SS
    last = None
    rt_post_us = []
    rt_post_ss = []
    for _, row in data.iterrows():
        if  last is not None and \
            last["ST_type"] == "text" and \
            row["ST_type"] != "text" and \
            row["RE_key"] == row["RE_true"]:
            if last["RE_time"] > 0:
                rt_post_us.append(row["RE_time"])
            else:
                rt_post_ss.append(row["RE_time"])

        last = row

    # mean RT post US
    beh_info.at[i, "mean RT post US"] = np.mean(rt_post_us)

    # SD RT post US
    beh_info.at[i, "SD RT post US"] = np.std(rt_post_us, ddof=1)

    # mean RT post SS
    beh_info.at[i, "mean RT post SS"] = np.mean(rt_post_ss)

    # SD RT post SS
    beh_info.at[i, "SD RT post SS"] = np.std(rt_post_ss, ddof=1)
        
    #-----------------------Zmienne związane z próbami ze Stopem----------------------#
    
    for s_t in stops_types:
        # mean SSD
        stops = data[data["ST_wait_time"] > 0]
        #stops = stops[stops["ST_name"].str.startswith(s_t, na=False)]
        stops = stops[stops["WORD_EMO"] == s_t]
        ssd = stops["ST_wait_time"].mean()
        beh_info.at[i, s_t + "_mean SSD"] = ssd
        
        us = stops[stops["RE_time"] > 0]
        beh_info.at[i, s_t + "_mean SSD US"] = us["ST_wait_time"].mean()
        # inhibition_rate
        ss = stops[stops["RE_key"].isnull()]
        
        ir = len(ss)/len(stops)*100
        beh_info.at[i, s_t + "_inhibition_rate"] = ir

        # bound_rt
        
        us_perc = 1 - ir/100
        rt_good_sorted = sorted(rt_good["RE_time"])
        bound_rt_index = round(len(rt_good_sorted) * us_perc)
        bound_rt = rt_good_sorted[bound_rt_index-1]
        beh_info.at[i, s_t + "_bound_rt"] = bound_rt

        # SSRT
        beh_info.at[i, s_t + "_SSRT"] = bound_rt - ssd

        # mean RT US
        us = stops[stops["RE_time"] > 0]
        beh_info.at[i, s_t + "_mean RT US"] = us["RE_time"].mean()

        # SD RT US
        beh_info.at[i, s_t + "_SD RT US"] = us["RE_time"].std()

        # mean RT post US, SD RT post US, mean RT post SS, SD RT post SS
        last = None
        rt_post_us = []
        rt_post_ss = []
        for _, row in data.iterrows():
            if  last is not None and \
                last["ST_type"] == "text" and \
                last["WORD_EMO"].startswith(s_t) and \
                row["ST_type"] != "text" and \
                row["RE_key"] == row["RE_true"]:
                if last["RE_time"] > 0:
                    rt_post_us.append(row["RE_time"])
                else:
                    rt_post_ss.append(row["RE_time"])

            last = row
        
        # mean RT post US
        beh_info.at[i, s_t + "_mean RT post US"] = np.mean(rt_post_us)

        # SD RT post US
        beh_info.at[i, s_t + "_SD RT post US"] = np.std(rt_post_us, ddof=1)

        # mean RT post SS
        beh_info.at[i, s_t + "_mean RT post SS"] = np.mean(rt_post_ss)

        # SD RT post SS
        beh_info.at[i, s_t + "_SD RT post SS"] = np.std(rt_post_ss, ddof=1)
    
# add means info for all participants
# beh_info.at[i+1, "names"] = "MEAN"
# for column in [c for c in columns_list if c !="names"]:
#     beh_info.at[i+1, column] = beh_info[column].mean()
    
beh_info

In [ ]:
beh_info.to_csv("beh_info_sst.csv")